# 01.1a: Extract Unembedding Matrix (γ)

**Goal:** Load Qwen3-4B-Instruct-2507, extract the unembedding matrix, and establish our coordinate system.

This notebook does three things:
1. Extract the unembedding matrix **γ** (151,936 × 2,560) from the model
2. Compute the centroid **μ** of the token cloud
3. Generate centered coordinates **γ'** = γ - μ

All three get saved to `data/tensors/` for use in future notebooks.

## Mathematical Background

The unembedding matrix **γ** maps the model's 2,560-dimensional hidden states to 151,936 logits (one per token). Each row of γ is a token's representation in **gamma space**—the natural Euclidean coordinates where the model actually lives.

We center the coordinates at the token cloud's centroid:

$$\mu = \frac{1}{N} \sum_{i=1}^{N} \gamma_i$$

$$\gamma' = \gamma - \mu$$

This puts our "telescope" at the galactic center, making angular measurements cleaner.

## Parameters

In [1]:
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
TENSOR_DIR = "../data/tensors"

## Imports

In [2]:
import torch
from transformers import AutoModelForCausalLM
from safetensors.torch import save_file
from pathlib import Path
import os

print("Imports loaded successfully.")

Imports loaded successfully.


## Setup: Create Output Directory

In [3]:
os.makedirs(TENSOR_DIR, exist_ok=True)
print(f"Tensor directory ready: {TENSOR_DIR}")

Tensor directory ready: ../data/tensors


## Step 1: Load Model and Extract Unembedding Matrix

In [4]:
print(f"Loading model: {MODEL_NAME}")
print("This will take a minute...\n")

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

print(f"Model loaded successfully.")
print(f"Model config vocab size: {model.config.vocab_size}")
print(f"Hidden size: {model.config.hidden_size}")

Loading model: Qwen/Qwen3-4B-Instruct-2507
This will take a minute...



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully.
Model config vocab size: 151936
Hidden size: 2560


In [5]:
# Extract the unembedding matrix (lm_head weights)
gamma = model.get_output_embeddings().weight.detach().clone()

print(f"\nExtracted γ (gamma):")
print(f"  Shape: {gamma.shape}")
print(f"  Dtype: {gamma.dtype}")
print(f"  Device: {gamma.device}")
print(f"  Memory: {gamma.element_size() * gamma.nelement() / 1024**2:.1f} MB")


Extracted γ (gamma):
  Shape: torch.Size([151936, 2560])
  Dtype: torch.float32
  Device: cpu
  Memory: 1483.8 MB


## Step 2: Compute Centroid

In [6]:
# Compute the centroid (mean position of all tokens)
mu = gamma.mean(dim=0)

print(f"Computed centroid μ:")
print(f"  Shape: {mu.shape}")
print(f"  Norm: {mu.norm().item():.4f} gamma units")
print(f"  Mean component: {mu.mean().item():.6f}")
print(f"  Std component: {mu.std().item():.6f}")

Computed centroid μ:
  Shape: torch.Size([2560])
  Norm: 0.3048 gamma units
  Mean component: -0.000026
  Std component: 0.006025


## Step 3: Center the Coordinates

In [7]:
# Center gamma at the centroid
gamma_centered = gamma - mu

print(f"\nCentered γ' (gamma_centered):")
print(f"  Shape: {gamma_centered.shape}")
print(f"  Mean norm: {gamma_centered.norm(dim=1).mean().item():.4f} gamma units")
print(f"  Std norm: {gamma_centered.norm(dim=1).std().item():.4f} gamma units")

# Verify centering (should be very close to zero)
new_mean = gamma_centered.mean(dim=0)
print(f"\nVerification (should be ~0):")
print(f"  Centered mean norm: {new_mean.norm().item():.2e}")


Centered γ' (gamma_centered):
  Shape: torch.Size([151936, 2560])
  Mean norm: 1.0401 gamma units
  Std norm: 0.1889 gamma units

Verification (should be ~0):
  Centered mean norm: 3.95e-08


## Step 4: Save Everything

In [8]:
# Save gamma (original unembedding matrix)
gamma_path = Path(TENSOR_DIR) / "gamma_qwen3_4b_instruct_2507.safetensors"
save_file({'gamma': gamma}, gamma_path)
print(f"Saved γ to: {gamma_path}")

# Save centroid
mu_path = Path(TENSOR_DIR) / "gamma_centroid_mu.safetensors"
save_file({'mu': mu}, mu_path)
print(f"Saved μ to: {mu_path}")

# Save centered gamma
gamma_centered_path = Path(TENSOR_DIR) / "gamma_centered_qwen3_4b_instruct_2507.safetensors"
save_file({'gamma_centered': gamma_centered}, gamma_centered_path)
print(f"Saved γ' to: {gamma_centered_path}")

Saved γ to: ../data/tensors/gamma_qwen3_4b_instruct_2507.safetensors
Saved μ to: ../data/tensors/gamma_centroid_mu.safetensors
Saved γ' to: ../data/tensors/gamma_centered_qwen3_4b_instruct_2507.safetensors


## Summary

Successfully extracted and centered the unembedding matrix. We now have:

- **γ** (gamma): The original 151,936 × 2,560 unembedding matrix
- **μ** (mu): The centroid of the token cloud in gamma space
- **γ'** (gamma_centered): Centered coordinates with our telescope at the galactic center

All future notebooks can load these tensors without touching the full model again.